# AADT Training

## Imports

In [1]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 12.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import os
import torch
import random
import cv2
from tqdm import tqdm
from matplotlib import pyplot as plt
import albumentations as album
from PIL import Image
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import random_split
from sklearn.metrics import accuracy_score
import plotly.express as px
import torchmetrics
from torchmetrics import MeanAbsolutePercentageError
from glob import glob
import plotly.graph_objs as go

%matplotlib inline

## Global Variables

In [3]:
COLAB = True

In [4]:
ROOT_DIR_PATH = os.path.abspath('..')

if COLAB:
  from google.colab import drive
  drive.mount('/content/drive')

  ROOT_DIR_PATH = os.path.abspath('drive/MyDrive/Spatial_Finance_Transport/minorRoads/')

VEHICLE_DETECTION_COUNT_PATH = os.path.join(ROOT_DIR_PATH, 'data/vehicle_counts_detection.csv')
AADT_PROCESSED_PATH = os.path.join(ROOT_DIR_PATH, 'data/ground_truth_data/aadt/processed/')

NN_MODEL_PATH = os.path.join(ROOT_DIR_PATH, "models/aadt_models/")

NORMALISED = True

Mounted at /content/drive


## Helper Functions

In [5]:
def zip_lists_by_name_attribute(list1, list2):
    """
    Zips two lists together based on the 'name' attribute of the elements.

    Args:
        list1 (list): The first list to be zipped.
        list2 (list): The second list to be zipped.

    Returns:
        list: The zipped list of tuples, where each tuple contains elements from list1 and list2
              that have the same 'name' attribute value.
    """
    # Create a dictionary to store the elements of list1 with the 'name' attribute as the key
    dict1 = {elem.name: elem for elem in list1}

    # Create a dictionary to store the elements of list2 with the 'name' attribute as the key
    dict2 = {elem.name: elem for elem in list2}

    # Get the set of keys that are common to both dictionaries
    common_keys = set(dict1.keys()) & set(dict2.keys())

    # Zip the common elements from list1 and list2 together
    zipped_list = [(dict1[key], dict2[key]) for key in common_keys]

    return zipped_list

## True Count Data
- Traffic monitoring stations for long-term traffic count data
    - Extract at same time as Satellite Image!
- How to use permanent and temporary traffic count stations

## Vehicle detection number
From vehicle detection model

## Road characteristics
From road characterstics pipeline

Includes:
- Road width
- Live speed data
- Directionality

## Data Loading

In [6]:
pattern = os.path.join(AADT_PROCESSED_PATH, 'aadt_*.csv')

processed_aadt_file_paths = [os.path.join(AADT_PROCESSED_PATH, os.path.basename(x)) for x in glob(pattern)]
print("Processed aadt files: {}".format(processed_aadt_file_paths))

processed_aadt_df_list = []
processed_aadt_df_test_list = []

for i in range(len(processed_aadt_file_paths)):
    processed_aadt_df = pd.read_csv(processed_aadt_file_paths[i])
    processed_aadt_df = processed_aadt_df.loc[:, ~processed_aadt_df.columns.str.contains('^Unnamed')]

    processed_aadt_df['site_name'] = processed_aadt_df['site_name'].astype(str)

    processed_aadt_df.name = processed_aadt_df.iloc[0]['Local Authority']+'_'+processed_aadt_df.iloc[0]['site_name'].replace('/', '_')

    if processed_aadt_df.iloc[0]['year'] == 2018:
        processed_aadt_df_test_list.append(processed_aadt_df)
    else:
        processed_aadt_df_list.append(processed_aadt_df)

Processed aadt files: ['/content/drive/MyDrive/Spatial_Finance_Transport/minorRoads/data/ground_truth_data/aadt/processed/aadt_Trafford_M60_9086B_2017.csv', '/content/drive/MyDrive/Spatial_Finance_Transport/minorRoads/data/ground_truth_data/aadt/processed/aadt_Blackburn with Darwen_30361033_2018.csv', '/content/drive/MyDrive/Spatial_Finance_Transport/minorRoads/data/ground_truth_data/aadt/processed/aadt_Luton_M1_2557A_2017.csv', '/content/drive/MyDrive/Spatial_Finance_Transport/minorRoads/data/ground_truth_data/aadt/processed/aadt_Blackburn with Darwen_30361033_2017.csv', '/content/drive/MyDrive/Spatial_Finance_Transport/minorRoads/data/ground_truth_data/aadt/processed/aadt_Havering_M25_5790A_2017.csv', '/content/drive/MyDrive/Spatial_Finance_Transport/minorRoads/data/ground_truth_data/aadt/processed/aadt_Havering_M25_5790B_2017.csv', '/content/drive/MyDrive/Spatial_Finance_Transport/minorRoads/data/ground_truth_data/aadt/processed/aadt_Blackburn with Darwen_30361032_2018.csv', '/conte

## Custom Dataset

In [7]:
class CustomDataset(Dataset):
    def __init__(self, df, normalised=True):

        self.name = df.name
        
        self.labels = torch.tensor(df['all_motor_vehicles'].values.astype('float32')).unsqueeze(1)
        self.labels_cars_and_taxis = torch.tensor(df['cars_and_taxis'].values.astype('float32')).unsqueeze(1)
        self.labels_buses_and_coaches = torch.tensor(df['buses_and_coaches'].values.astype('float32')).unsqueeze(1)
        self.labels_lgvs = torch.tensor(df['lgvs'].values.astype('float32')).unsqueeze(1)
        self.labels_all_hgvs = torch.tensor(df['all_hgvs'].values.astype('float32')).unsqueeze(1)
        
        #self.speed_data = pd.read_csv(SPEED_DATA_PATH) 
        #self.road_width = pd.read_csv(ROAD_WIDTH_PATH)
        self.hour = torch.tensor(df['hour'].values.astype('float32')).unsqueeze(1)
        self.avg_mph = torch.tensor(df['avg_mph'].values.astype('float32')).unsqueeze(1)
        self.day = torch.tensor(df['day'].values.astype('float32')).unsqueeze(1)
        self.month = torch.tensor(df['month'].values.astype('float32')).unsqueeze(1)

        if normalised:
            vehicle_types = ['0-520cm_normalised', '521-660cm_normalised', '661-1160cm_normalised', '1160+cm_normalised', 'total_volume_normalised']
        else:
            vehicle_types = ['0-520cm', '521-660cm', '661-1160cm', '1160+cm', 'total_volume']

        self.small_vehicle = torch.tensor(df[vehicle_types[0]].values.astype('float32')).unsqueeze(1)
        self.mid_vehicle = torch.tensor(df[vehicle_types[1]].values.astype('float32')).unsqueeze(1)
        self.large_vehicle = torch.tensor(df[vehicle_types[2]].values.astype('float32')).unsqueeze(1)
        self.very_large_vehicle = torch.tensor(df[vehicle_types[3]].values.astype('float32')).unsqueeze(1)
        self.vehicle_count = torch.tensor(df[vehicle_types[4]].values.astype('float32')).unsqueeze(1)

        self.x = torch.concat((self.vehicle_count, self.small_vehicle, self.mid_vehicle, self.large_vehicle, self.very_large_vehicle, self.avg_mph, self.day, self.month, self.hour), dim=-1)
        self.y = torch.concat((self.labels, self.labels_cars_and_taxis, self.labels_buses_and_coaches, self.labels_lgvs, self.labels_all_hgvs), dim=-1)

        # Reshape y to have shape (batch_size, 5)
        self.y = self.y.view(-1, 5)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.name, self.x[idx], self.y[idx]

In [8]:
dataset_list = []

for df in processed_aadt_df_list:

    custom_dataset = CustomDataset(df, normalised=NORMALISED)
    dataset_list.append(custom_dataset)

## Neural Network Models

In [9]:
class NeuralNetwork(nn.Module):
    def __init__(self, name):
        super(NeuralNetwork, self).__init__()

        self.name = name
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(9, 7),
            nn.Linear(7,7),
            nn.LeakyReLU(negative_slope=0.2),
            nn.Linear(7,5),
            nn.ReLU()
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [10]:
nn_model_list = []

for df in processed_aadt_df_list:
    nn_model = NeuralNetwork(name=df.name)
    nn_model_list.append(nn_model)

nn_model_list[0]

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=9, out_features=7, bias=True)
    (1): Linear(in_features=7, out_features=7, bias=True)
    (2): LeakyReLU(negative_slope=0.2)
    (3): Linear(in_features=7, out_features=5, bias=True)
    (4): ReLU()
  )
)

## EarlyStopping

In [11]:
class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = np.inf

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

## Weights Initialisation

In [12]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.ones_(m.weight)
        m.bias.data.fill_(0.01)

for nn_model in nn_model_list:
    nn_model.apply(init_weights)

## HyperParameters

In [13]:
learning_rate = 0.01
batch_size = 256
epochs = 3
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(nn_model.parameters(), lr=learning_rate)

MAPE = MeanAbsolutePercentageError()

## DataLoaders

In [14]:
train_dataloader_list = []
val_dataloader_list = []

for dataset in dataset_list:
    train_split = 0.8
    train_data, val_data = random_split(dataset, [train_split, 1-train_split])

    train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=False, sampler=None,
                    batch_sampler=None, num_workers=0, collate_fn=None,
                    pin_memory=False, drop_last=False, timeout=0,
                    worker_init_fn=None, prefetch_factor=None,
                    persistent_workers=False)
    
    val_dataloader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=False, sampler=None,
                    batch_sampler=None, num_workers=0, collate_fn=None,
                    pin_memory=False, drop_last=False, timeout=0,
                    worker_init_fn=None, prefetch_factor=None,
                    persistent_workers=False)
    
    train_dataloader_list.append(train_dataloader)
    val_dataloader_list.append(val_dataloader)

## Training & Validation

In [15]:
def run_epoch(ep_id, action, loader, model, optimizer, criterion, early_stopper):
    losses = [] # Keep list of accuracies to track progress
    is_training = action == "train" # True when action == "train", else False 

    # Looping over all batches
    for batch_idx, batch in enumerate(loader): 
        dl_name, x, y = batch

        # Assert we are training the correct model with the correct dataset!
        assert dl_name[0] == model.name

        # Resetting the optimizer gradients
        optimizer.zero_grad()

        # Setting model to train or test
        with torch.set_grad_enabled(is_training):
            
            # Feed batch to model
            logits = model(x)

            # Determine the batch size and reshape logits and y accordingly
            batch_size = logits.size(0)
            logits = logits.view(batch_size, -1)
            y = y.view(batch_size, -1)

            #print("y shape: {}".format(y.shape))
            #print("logits shape: {}".format(logits.shape))
            #print("y values: {}".format(y))
            #print("logits values: {}".format(logits))

            # Calculate the loss based on predictions and real labels
            loss = criterion(logits, y)

            #print("loss: {}".format(loss.item()))
            mape_loss = MAPE(logits, y)

            # If training, perform backprop and update weights
            if is_training:
                loss.backward()
                optimizer.step()

            # Append current batch accuracy
            losses.append(mape_loss.detach().numpy())

            # Print some stats every 50th batch 
            if batch_idx % 50 == 0:
                print(f"{action.capitalize()}, Epoch: {ep_id+1}, Batch {batch_idx}: Loss = {loss.item()}")

        if not is_training:
            if early_stopper.early_stop(mape_loss.detach().numpy()):
                print("Entered Early Stopping")             
                break
                    
    # Return accuracies to main loop                 
    return losses

In [16]:
def train(epochs, train_dl, val_dl, model, optimizer, criterion, early_stopper):

    # Keep lists of accuracies to track performance on train and test sets
    train_losses = []
    val_losses = []

    # Looping over epochs
    for epoch in range(epochs):
        
        # Looping over train set and training
        train_loss = run_epoch(epoch, "train", train_dl, model, optimizer, criterion, early_stopper=early_stopper)

        # Looping over test set
        val_loss = run_epoch(epoch, "val", val_dl, model, optimizer, criterion, early_stopper=early_stopper) 

        # Collecting stats
        train_losses += train_loss
        val_losses += val_loss         
            
    return train_losses, val_losses

In [17]:
def main(epochs, train_dl_list, val_dl_list, model_list):
    all_train_losses = []
    all_val_losses = []

    for (model, train_dl, val_dl) in zip(model_list, train_dl_list, val_dl_list):

        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

        loss_fn = nn.MSELoss()

        early_stopper = EarlyStopper(patience=3, min_delta=10)
        
        train_losses, val_losses = train(epochs, train_dl, val_dl, model, optimizer=optimizer, criterion=loss_fn, early_stopper=early_stopper)

        all_train_losses.append((model.name, train_losses))
        all_val_losses.append((model.name, val_losses))

    return all_train_losses, all_val_losses

In [18]:
all_train_losses, all_val_losses = main(epochs=epochs, train_dl_list=train_dataloader_list, val_dl_list=val_dataloader_list, model_list=nn_model_list)

Train, Epoch: 1, Batch 0: Loss = 20018116.0
Train, Epoch: 1, Batch 50: Loss = 1069315.25
Val, Epoch: 1, Batch 0: Loss = 1027028.875
Train, Epoch: 2, Batch 0: Loss = 1000725.875
Train, Epoch: 2, Batch 50: Loss = 340640.8125
Val, Epoch: 2, Batch 0: Loss = 343305.5
Train, Epoch: 3, Batch 0: Loss = 335974.75
Train, Epoch: 3, Batch 50: Loss = 197186.84375
Val, Epoch: 3, Batch 0: Loss = 198257.5625
Train, Epoch: 1, Batch 0: Loss = 19361372.0
Train, Epoch: 1, Batch 50: Loss = 1249521.625
Val, Epoch: 1, Batch 0: Loss = 1203069.0
Entered Early Stopping
Train, Epoch: 2, Batch 0: Loss = 1139443.875
Train, Epoch: 2, Batch 50: Loss = 335746.8125
Val, Epoch: 2, Batch 0: Loss = 335442.40625
Entered Early Stopping
Train, Epoch: 3, Batch 0: Loss = 316805.4375
Train, Epoch: 3, Batch 50: Loss = 149757.03125
Val, Epoch: 3, Batch 0: Loss = 151347.703125
Entered Early Stopping
Train, Epoch: 1, Batch 0: Loss = 20695258.0
Train, Epoch: 1, Batch 50: Loss = 1231606.25
Val, Epoch: 1, Batch 0: Loss = 1238904.75
E

### Loss Curves Plot

In [19]:
for model_train_losses in all_train_losses:
    (name, train_losses) = model_train_losses
    
    fig = px.line(train_losses, title="Train MAPE Loss Curve: "+name)

    fig.update_layout(
            autosize=False,
            width=1200,
            height=750,
            font=dict(size=20),
            )

    fig.show()

In [20]:
for model_val_losses in all_val_losses:
    (name, val_losses) = model_val_losses
    
    fig = px.line(val_losses, title="Validation MAPE Loss Curve: "+name, color_discrete_sequence=['red'])

    fig.update_layout(
        autosize=False,
        width=1200,
        height=750,
        font=dict(size=20),
        )
    
    fig.show()

### Random val samples

In [21]:
for nn_model, val_data in zip(nn_model_list, dataset_list):    
    
    preds = []
    ground_truth = []
    vehicle_counts = []


    for i in range(10):
        vehicle_count = 0
        random_idx = np.random.randint(0,len(val_data))
        dataset_name, x, y = val_data[random_idx]

        #print("x shape: {}".format(x.shape))
        #print("y shape: {}".format(y.shape))
        
        assert dataset_name == nn_model.name

        vehicle_count = x[0]
        vehicle_counts.append(vehicle_count)
        ground_truth.append(float(y[0]))
        pred_y = nn_model(x).detach().numpy()
        #print("pred_y shape: {}".format(pred_y.shape))
        preds.append(pred_y[0])

    df = pd.DataFrame({'vehicle_count': vehicle_counts, 'ground_truth': ground_truth, 'predictions': preds})

    fig = px.scatter(df, x='vehicle_count', y=['ground_truth', 'predictions'], title=nn_model.name)

    fig.show()

## Test Set

In [22]:
test_dataset_list = []

for df in processed_aadt_df_test_list:

    custom_dataset = CustomDataset(df, normalised=NORMALISED)
    test_dataset_list.append(custom_dataset)

In [23]:
test_dataloader_list = []

for test_data in test_dataset_list:
    test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=False, sampler=None,
                    batch_sampler=None, num_workers=0, collate_fn=None,
                    pin_memory=False, drop_last=False, timeout=0,
                    worker_init_fn=None, prefetch_factor=None,
                    persistent_workers=False)
    
    test_dataloader_list.append(test_dataloader)

In [24]:
def run_epoch_test(loader, model_list):
    losses = [] # Keep list of accuracies to track progress

    correct_model = None

    # Looping over all batches
    with torch.no_grad():
        for batch_idx, batch in enumerate(loader):
            dl_name, x, y = batch

            # find right model for dataset
            for model in model_list:
                if model.name == dl_name[0]:
                    correct_model = model
                    break

            # Setting model to evaluation mode
            correct_model.eval()

            # Assert we are testing the correct model with the correct dataset!
            assert dl_name[0] == model.name

            # Feed batch to model
            logits = correct_model(x).squeeze(1)

            mape_loss = MAPE(logits, y)

            # Append current batch accuracy
            losses.append(mape_loss.detach().numpy())

    # Return accuracies to main loop                 
    return correct_model, losses


In [25]:
def main_test(test_dl_list, model_list):
    all_test_losses = []

    for test_dl in test_dl_list:

        model, test_loss = run_epoch_test(test_dl, model_list)
        
        all_test_losses.append((model.name, test_loss))

    return all_test_losses

In [26]:
#all_test_losses = main_test(test_dataloader_list, nn_model_list)

In [27]:
"""for model_test_losses in all_test_losses:
    (name, test_losses) = model_test_losses
    
    fig = px.line(test_losses, title="Test MAPE Loss Curve: "+name)

    fig.update_layout(
            autosize=False,
            width=1200,
            height=750,
            font=dict(size=20),
            )

    fig.show()
"""

'for model_test_losses in all_test_losses:\n    (name, test_losses) = model_test_losses\n    \n    fig = px.line(test_losses, title="Test MAPE Loss Curve: "+name)\n\n    fig.update_layout(\n            autosize=False,\n            width=1200,\n            height=750,\n            font=dict(size=20),\n            )\n\n    fig.show()\n'

### Visualise Random Test Samples

In [28]:
"""for test_data in test_dataset_list:    
    
    preds = []
    ground_truth = []
    vehicle_counts = []

    model = None

    # find right model for dataset
    for i in range(len(nn_model_list)):
        if nn_model_list[i].name == test_data.name:
            model = nn_model_list[i]
            break

    for i in range(10):
        vehicle_count = 0
        random_idx = np.random.randint(0,len(test_data))
        dataset_name, x, y = test_data[random_idx]

        assert dataset_name == model.name

        vehicle_count = x[0]
        vehicle_counts.append(vehicle_count)
        ground_truth.append(float(y))
        pred_y = float(model(x)[0])
        preds.append(pred_y)

    df = pd.DataFrame({'vehicle_count': vehicle_counts, 'ground_truth': ground_truth, 'predictions': preds})

    fig = px.scatter(df, x='vehicle_count', y=['ground_truth', 'predictions'], title=model.name)

    fig.show()
"""

"for test_data in test_dataset_list:    \n    \n    preds = []\n    ground_truth = []\n    vehicle_counts = []\n\n    model = None\n\n    # find right model for dataset\n    for i in range(len(nn_model_list)):\n        if nn_model_list[i].name == test_data.name:\n            model = nn_model_list[i]\n            break\n\n    for i in range(10):\n        vehicle_count = 0\n        random_idx = np.random.randint(0,len(test_data))\n        dataset_name, x, y = test_data[random_idx]\n\n        assert dataset_name == model.name\n\n        vehicle_count = x[0]\n        vehicle_counts.append(vehicle_count)\n        ground_truth.append(float(y))\n        pred_y = float(model(x)[0])\n        preds.append(pred_y)\n\n    df = pd.DataFrame({'vehicle_count': vehicle_counts, 'ground_truth': ground_truth, 'predictions': preds})\n\n    fig = px.scatter(df, x='vehicle_count', y=['ground_truth', 'predictions'], title=model.name)\n\n    fig.show()\n"

## Save Model

In [29]:
for nn_model in nn_model_list:

  print(nn_model.name)

  if 'Blackburn with Darwen' in nn_model.name:
    nn_model.name = nn_model.name.replace('Blackburn with Darwen', 'blackburn')

  torch.save(nn_model.state_dict(), NN_MODEL_PATH+"nn_model_{}.pth".format(nn_model.name.lower()))

Trafford_M60_9086B
Luton_M1_2557A
Blackburn with Darwen_30361033
Havering_M25_5790A
Havering_M25_5790B
Blackburn with Darwen_30361032
Luton_M1_2557B
Hounslow_M4_2188A
Hounslow_M4_2188B
Trafford_M60_9083A
